# Deneme 1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [ ]:
# 1. Veri Hazırlığı
def xox_veri_hazirla(num_samples=1000):
    """
    Rastgele XOX tahtaları ve sonuçlar oluşturur.
    """
    X = np.random.randint(-1, 2, (num_samples, 3, 3, 1))  # 3x3 tahtalar
    y = []
    for tahta in X:
        # Basit bir kazanan belirleyici
        if np.any(np.sum(tahta, axis=0) == 3) or np.any(np.sum(tahta, axis=1) == 3):
            y.append(1)  # X kazandı
        elif np.any(np.sum(tahta, axis=0) == -3) or np.any(np.sum(tahta, axis=1) == -3):
            y.append(2)  # O kazandı
        elif np.trace(tahta) == 3 or np.trace(np.fliplr(tahta)) == 3:
            y.append(1)  # X kazandı
        elif np.trace(tahta) == -3 or np.trace(np.fliplr(tahta)) == -3:
            y.append(2)  # O kazandı
        else:
            y.append(0)  # Berabere
    return X, to_categorical(y, 3)


In [ ]:
# Veriyi oluştur
X, y = xox_veri_hazirla(num_samples=100000)


In [ ]:
# Eğitim ve test verisini ayır
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 2. Model Oluşturma
model = models.Sequential([
    layers.Conv2D(32, (2, 2), activation='relu', input_shape=(3, 3, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 sınıf: X kazandı, O kazandı, berabere
])

In [ ]:
# Modeli derleme
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 3. Modeli Eğitme
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6775 - loss: 0.7275 - val_accuracy: 0.7354 - val_loss: 0.5888
Epoch 2/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7431 - loss: 0.5788 - val_accuracy: 0.7820 - val_loss: 0.5166
Epoch 3/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7774 - loss: 0.5121 - val_accuracy: 0.7890 - val_loss: 0.4886
Epoch 4/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8025 - loss: 0.4663 - val_accuracy: 0.8176 - val_loss: 0.4348
Epoch 5/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8142 - loss: 0.4359 - val_accuracy: 0.8209 - val_loss: 0.4272
Epoch 6/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8296 - loss: 0.4064 - val_accuracy: 0.8380 - val_loss: 0.3816
Epoch 7/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8413 - loss: 0.3805 - val_accuracy: 0.8514 - val_loss: 0.3633
Epoch 8/20
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8456 - loss: 0.3664 - 

In [ ]:
# Modeli kaydet
model.save("xox_model.h5")

In [ ]:
# 4. Model Yükleme ve Test Etme
from tensorflow.keras.models import load_model

# Eğitilmiş modeli yükleme
model = load_model("xox_model.h5")

In [ ]:
# Test verisinde doğruluk
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Doğruluğu: {test_acc:.2f}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8996 - loss: 0.2656
Test Doğruluğu: 0.89


In [ ]:
# 5. XOX Tahtası ile Tahmin Yapma
def tahmin_yap(model, tahta):
    """
    Modelin bir XOX tahtasında tahmin yapmasını sağlar.
    """
    tahta = np.array(tahta).reshape(1, 3, 3, 1)
    tahmin = model.predict(tahta)
    sonuc = np.argmax(tahmin)

    if sonuc == 0:
        return "Oyun Berabere."
    elif sonuc == 1:
        return "X Kazandı!"
    elif sonuc == 2:
        return "O Kazandı!"

In [ ]:
# Örnek XOX tahtası
xox_tahtasi = [
    [1, 0, 0],
    [0, 1, -1],
    [-1, 0, 1]
]

In [ ]:
# Tahmin yap
sonuc = tahmin_yap(model, xox_tahtasi)
print("Tahmin:", sonuc)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Tahmin: X Kazandı!


# Deneme 2

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

# Tahta durumları
EMPTY = 0
PLAYER_X = 1
PLAYER_O = -1

# Basit CNN modelini oluşturma (MaxPooling kaldırıldı)
def build_simple_cnn_model():
    model = models.Sequential([
        layers.Conv2D(32, (2, 2), activation='relu', input_shape=(3, 3, 1)),
        layers.Conv2D(64, (1, 1), activation='relu'),
        layers.Conv2D(128, (1, 1), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(9, activation='softmax')  # 9 olasılık, 3x3 tahta
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Veri setini oluşturma
def generate_training_data(num_samples=100000):
    X = []
    y = []
    for _ in range(num_samples):
        board = np.random.choice([EMPTY, PLAYER_X, PLAYER_O], size=(3, 3))
        valid_moves = np.argwhere(board == EMPTY)
        if len(valid_moves) > 0:
            move = random.choice(valid_moves)
            label = np.zeros(9)
            label[move[0] * 3 + move[1]] = 1
            X.append(board.reshape(3, 3, 1))  # Kanal boyutunu ekle
            y.append(label)
    return np.array(X), np.array(y)

# Eğitim verisini oluştur
X_train, y_train = generate_training_data(num_samples=500000)

# Veri artırma (augmentation)
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

# Modeli oluştur ve eğit
model = build_simple_cnn_model()
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=30, verbose=1)

# Modeli kaydet
model.save("xox_model_simple_cnn.h5")

# En iyi hamleyi seçme
def en_iyi_hamle(model, tahta):
    tahta = tahta.reshape(1, 3, 3, 1)
    predictions = model.predict(tahta).reshape(3, 3)
    bos_hucreler = np.argwhere(tahta.reshape(3, 3) == EMPTY)
    en_iyi = max(bos_hucreler, key=lambda x: predictions[x[0], x[1]])
    return en_iyi

# Oyun mekaniği
def yazdir_tahta(tahta):
    semboller = {EMPTY: ".", PLAYER_X: "X", PLAYER_O: "O"}
    print("\n".join([" ".join([semboller[tahta[i][j]] for j in range(3)]) for i in range(3)]))

def kontrol_kazanan(tahta):
    for i in range(3):
        if abs(sum(tahta[i, :])) == 3: return tahta[i, 0]
        if abs(sum(tahta[:, i])) == 3: return tahta[0, i]
    if abs(tahta.trace()) == 3: return tahta[0, 0]
    if abs(np.fliplr(tahta).trace()) == 3: return tahta[0, 2]
    if not (tahta == EMPTY).any(): return 0
    return None

def oyuncu_hamlesi(tahta):
    while True:
        try:
            hamle = input("Hamleni yap (satır, sütun): ")
            satir, sutun = map(int, hamle.split(","))
            if tahta[satir, sutun] == EMPTY:
                return satir, sutun
            else:
                print("Bu hücre dolu!")
        except:
            print("Geçersiz giriş. Tekrar dene.")

def oyun():
    tahta = np.zeros((3, 3), dtype=int)
    oyuncu = PLAYER_X
    while True:
        yazdir_tahta(tahta)
        kazanan = kontrol_kazanan(tahta)
        if kazanan is not None:
            if kazanan == PLAYER_X:
                print("Tebrikler, X kazandı!")
            elif kazanan == PLAYER_O:
                print("O kazandı!")
            else:
                print("Oyun berabere!")
            break

        if oyuncu == PLAYER_X:
            print("Sıra insan oyuncuda!")
            satir, sutun = oyuncu_hamlesi(tahta)
        else:
            print("Sıra AI oyuncuda!")
            satir, sutun = en_iyi_hamle(model, tahta)

        tahta[satir, sutun] = oyuncu
        oyuncu = PLAYER_O if oyuncu == PLAYER_X else PLAYER_X

# Eğitim sonrası tahmin raporu
def performans_analizi(X_test, y_test):
    predictions = model.predict(X_test)
    y_true = y_test.argmax(axis=1)
    y_pred = predictions.argmax(axis=1)
    print(classification_report(y_true, y_pred))

# Oyun başlatma
oyun()



Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


    8/15216 ━━━━━━━━━━━━━━━━━━━━ 3:58 16ms/step - accuracy: 0.1322 - loss: 2.2010

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15216/15216 ━━━━━━━━━━━━━━━━━━━━ 204s 13ms/step - accuracy: 0.3524 - loss: 1.4091
Epoch 2/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 200s 13ms/step - accuracy: 0.3778 - loss: 1.2155
Epoch 3/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 199s 13ms/step - accuracy: 0.3808 - loss: 1.1990
Epoch 4/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 199s 13ms/step - accuracy: 0.3835 - loss: 1.1875
Epoch 5/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 203s 13ms/step - accuracy: 0.3816 - loss: 1.1851
Epoch 6/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 199s 13ms/step - accuracy: 0.3827 - loss: 1.1814
Epoch 7/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 205s 13ms/step - accuracy: 0.3845 - loss: 1.1803
Epoch 8/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 261s 13ms/step - accuracy: 0.3842 - loss: 1.1795
Epoch 9/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.3858 - loss: 1.1789
Epoch 10/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 201s 13ms/step - accuracy: 0.3846 - loss: 1.1798
Epoch 11/30
15216/15216 ━━━━━━━━━━━━━━━━━━━━ 201s 13ms/step - accuracy: 0.3852 -

. . .
. . .
. . .
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 1,1
. . .
. X .
. . .
Sıra AI oyuncuda!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
. . .
. X O
. . .
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 2,2
. . .
. X O
. . X
Sıra AI oyuncuda!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
. . .
O X O
. . X
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 0
Geçersiz giriş. Tekrar dene.
Hamleni yap (satır, sütun): 0,0
X . .
O X O
. . X
Tebrikler, X kazandı!


In [ ]:
oyun()

. . .
. . .
. . .
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 0,0
X . .
. . .
. . .
Sıra AI oyuncuda!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
X . .
. O .
. . .
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 0,1
X X .
. O .
. . .
Sıra AI oyuncuda!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
X X .
. O O
. . .
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 1,0
X X .
X O O
. . .
Sıra AI oyuncuda!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
X X O
X O O
. . .
Sıra insan oyuncuda!
Hamleni yap (satır, sütun): 1,2
Bu hücre dolu!
Hamleni yap (satır, sütun): 2,1
X X O
X O O
. X .
Sıra AI oyuncuda!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
X X O
X O O
O X .
O kazandı!
